In [3]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import pandas as pd 

In [4]:
player_dict = players.get_players()
kobe = [player for player in player_dict if player['full_name'] == 'Kobe Bryant'][0]
kobe_id = kobe['id']

In [5]:
gamelog_kobe = playergamelog.PlayerGameLog(player_id=kobe_id, season = SeasonAll.all)

In [6]:
df_kobe_games_all = gamelog_kobe.get_data_frames()[0]

In [7]:
df_kobe_games_all['OPPONENT'] = df_kobe_games_all['MATCHUP'].apply(lambda x: x.split()[-1])

In [8]:
# clean up opponent abbreviation
remap = {"UTH": "UTA", "SAN": "SAS", "PHO": "PHX",  "NJN": "BKN", "NOK": "NOP", "NOH":"NOP", "SEA": "OKC", "CHH": "CHA", "VAN": "MEM"}
df_kobe_games_all['OPPONENT'].replace(remap, inplace=True)
df_kobe_games_all['OPPONENT'].value_counts()

LAC    69
MEM    67
GSW    67
DEN    66
OKC    65
PHX    64
SAC    64
MIN    64
POR    62
SAS    61
HOU    61
UTA    60
DAL    60
NOP    41
IND    34
CLE    34
NYK    34
TOR    33
MIA    33
DET    33
WAS    32
MIL    32
PHI    32
CHI    31
BOS    31
CHA    31
ORL    29
ATL    29
BKN    27
Name: OPPONENT, dtype: int64

In [9]:
df_kobe_games_all.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'OPPONENT'],
      dtype='object')

In [10]:
out_df = df_kobe_games_all.groupby('OPPONENT').agg(
    GAME_COUNT = ('Game_ID', 'count'),
    W_COUNT = ('WL', lambda x: (x=='W').sum()),
    L_COUNT = ('WL', lambda x: (x=='L').sum()),
    MIN_TOTAL = ('MIN', 'sum'),
    MIN_PER = ('MIN', 'mean'),
    FGM_TOTAL = ('FGM', 'sum'),
    FGM_PER = ('FGM', 'mean'),
    FGA_TOTAL = ('FGA', 'sum'),
    FGA_PER=('FGA', 'mean'),
    
    
    
    ).round(1)


In [11]:
out_df 

,GAME_COUNT,W_COUNT,L_COUNT,MIN_TOTAL,MIN_PER,FGM_TOTAL,FGM_PER,FGA_TOTAL,FGA_PER
OPPONENT,,,,,,,,,
ATL,29,18,11,987,34.0,235,8.1,519,17.9
BKN,27,20,7,991,36.7,214,7.9,505,18.7
BOS,31,18,13,1146,37.0,273,8.8,652,21.0
CHA,31,19,12,1100,35.5,266,8.6,620,20.0
CHI,31,18,13,1112,35.9,268,8.6,610,19.7
CLE,34,17,17,1169,34.4,274,8.1,621,18.3
DAL,60,42,18,2222,37.0,499,8.3,1116,18.6
DEN,66,42,24,2311,35.0,568,8.6,1243,18.8
DET,33,22,11,1161,35.2,266,8.1,602,18.2


In [12]:
out_df.T.to_json("../dist/assets/data/game_stats.json")

In [13]:
out_df.T.to_csv("../dist/assets/data/game_stats.csv")